In [192]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold #Кросс-валидация
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import os, codecs

In [198]:
X = []
y = []
read = codecs.open("data.txt", 'r', 'utf-8')
for i in (read):
    X.append(i) 
read.close()

read = open("data_target.txt", 'r')
for i in (read):
    y.append(i) 
read.close()


y = [int(y[i]) for i in range(len(y))]
X = [X[i].lower() for i in range(len(X))]  

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=241)


In [194]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
vectorizer.get_feature_names()

clf = SVC(C=100, kernel='linear', random_state=241) #С=1.0 по умолчанию
clf.fit(X_train,y_train)

#10 слов с наибольшим абсолютным значением веса
#Находим слова с наиболее большим весом из get_feature_names()
df = pd.DataFrame(np.transpose(abs(clf.coef_.toarray())), #Берем по модулю потому что 2 класса todense() переводит матрицу из разряженной в обычную 
                   index=np.asarray(vectorizer.get_feature_names()), 
                   columns=["col"])


### Вывод опорных слов (векторов)

In [195]:
df_sort = df.sort_values(by='col')[::-1]
display(df_sort[:20].sort_index())

,col
10,0.603934
больницу,0.825788
вирус,0.580652
если,0.594214
заболевания,0.611246
инфекцией,0.607513
китае,0.587685
коронавирус,1.534494
коронавируса,0.962408
мест,0.603934


### Вывод и предсказание

In [196]:
X_test = vectorizer.transform(X_test)
print(clf.score(X_test, y_test)) #оценка 0.85
print(accuracy_score(y_test, clf.predict(X_test)))
print(f1_score(y_test, clf.predict(X_test))) # оценка 0.5
print(precision_score(y_test, clf.predict(X_test)))
print(recall_score(y_test, clf.predict(X_test)))

X_test2 = ['В Москве на 3 марта новых случаев заболевания коронавирусом не выявлено', 
          'В таблицах ниже приведены подтвержденные случаи заболевания коронавирусом (2019-nCoV, официально известный как SARS-CoV-2 или COVID-19) в Китае и других странах.Чтобы просмотреть карту распределения и временную шкалу, прокрутите вниз',
          'Вакцина против коронавируса: насколько быстро её могут создать?',
          'Ранее сообщалось, что на борту воздушного судна находятся 113 пассажиров и шесть членов экипажа',
           'У двух россиян в ОАЭ выявили коронавирус, сообщило агентство WAM со ссылкой на Минздрав страны. В ведомстве отметили, что заболевшие контактировали с итальянскими велогонщиками, которые были заражены вирусом.'
          ]
X_test2 = vectorizer.transform(X_test2)
clf.predict(X_test2)

clf.score(X_test2, [1, 1, 1, 0, 1])

0.8518518518518519
0.8518518518518519
0.5
0.5
0.5


0.8